In [2]:
import syft as sy
from syft import make_plan
from syft.lib.python.string import String
from syft import logger
import torch

sy.load('transformers')

# Create client and test string
alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_client()

text = "This is a test."
text_ptr = String(text).send(alice_client)

[2021-05-27T18:08:18.039728+0200][CRITICAL][logger]][17606] You tried to deserialize an unsupported type. This can be caused by several reasons. Either you are actively writing Syft code and forgot to create one, or you are trying to deserialize an object which was serialized using a different version of Syft and the object you tried to deserialize is not supported in this version.


TypeError: You tried to deserialize an unsupported type. This can be caused by several reasons. Either you are actively writing Syft code and forgot to create one, or you are trying to deserialize an object which was serialized using a different version of Syft and the object you tried to deserialize is not supported in this version.

In [2]:
# Load Tokenizer and Model
from transformers import AutoTokenizer

model_name = 'distilbert-base-uncased'
hf_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [3]:
# syft relative
from syft import serialize, deserialize
from syft.lib.python.dict import Dict
from syft import serialize, deserialize
from syft.proto.lib.python.dict_pb2 import Dict as Dict_PB
from syft.generate_wrapper import GenerateWrapper

# Third party
from transformers import BatchEncoding

def object2proto(obj: object) -> Dict_PB:
    # Wrap in sy Dict to make nested serialization easy.
    obj_dict = Dict(obj.data)
    return serialize(obj_dict)


def proto2object(proto: Dict_PB) -> BatchEncoding:
    obj_dict = deserialize(proto).data
    return BatchEncoding(data=obj_dict)

GenerateWrapper(
    wrapped_type=BatchEncoding,
    import_path="transformers.BatchEncoding",
    protobuf_scheme=Dict_PB,
    type_object2proto=object2proto,
    type_proto2object=proto2object,
)


In [4]:
# Test for list
be_list = hf_tokenizer(["test", "test2"], padding=True, truncation=True, max_length=100)

assert proto2object(object2proto(be_list)) == be_list

# Test for tensor
be_tensor = hf_tokenizer(["test", "test2"], padding=True, return_tensors='pt', truncation=True, max_length=100)

be_tensor_deserialized = proto2object(object2proto(be_tensor))
for k, v in be_tensor.items():
    assert torch.eq(v, be_tensor_deserialized[k]).all()

[2021-05-27T17:23:56.040025+0200][CRITICAL][logger]][17382] You tried to deserialize an unsupported type. This can be caused by several reasons. Either you are actively writing Syft code and forgot to create one, or you are trying to deserialize an object which was serialized using a different version of Syft and the object you tried to deserialize is not supported in this version.


TypeError: You tried to deserialize an unsupported type. This can be caused by several reasons. Either you are actively writing Syft code and forgot to create one, or you are trying to deserialize an object which was serialized using a different version of Syft and the object you tried to deserialize is not supported in this version.